In [46]:
import pandas as pd
df_=pd.read_csv('tea_demand.csv')
df_.head(3)

,brand_id\tdate\treceipts_count
0,5152\t2016-07-01\t98
1,178\t2016-07-01\t234
2,29\t2016-07-01\t622


In [47]:
cols=df_.columns
cols=cols[0].split('\t')
cols

['brand_id', 'date', 'receipts_count']

In [49]:
df=pd.DataFrame(columns=['brand_id', 'date', 'receipts_count'])
df[['brand_id', 'date', 'receipts_count']]=df_.apply(lambda x: pd.Series(x.iloc[0].split('\t')),axis=1)
df.head(3)


,brand_id,date,receipts_count
0,5152,2016-07-01,98
1,178,2016-07-01,234
2,29,2016-07-01,622


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459 entries, 0 to 458
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   brand_id        459 non-null    object
 1   date            459 non-null    object
 2   receipts_count  459 non-null    object
dtypes: object(3)
memory usage: 10.9+ KB
